<h1><i>Your Road</i></h1>

<h2>Assumption</h2>
<p> <b>First</b>,assume user profile Dataframe filled with the most top 10 categories of venues which prefered by user.</p><p><b>Second</b>,assume a coordinates of the stop points in the available roads for the trip between <b> Vancouver, CA</b> and <b> Los Angeles, US</b>.</p>

<h1>Data preparation</h1>

In [2]:
import pandas as pd
import numpy as np


Get Coordinates of the each stop stations for the expected trip between <b> Vancouver,CA</b> and <b> Los Angeles</b> 

In [3]:
van_los={"start":["Vancouver,CA","Vancouver,CA"],
         "stp1":["Seattle Washington,USA ","Seattle Washington,USA "],
         "stp2":["Portland Oregon,USA ","Kennewick Washington,USA"],
         "stp3":["Medford Oregon,USA ","Twin Falls Idaho,USA "],
         "stp4":["Sacramento California,USA ","Las Vegas Nevada,USA "],
         "Final":["Los Angeles California,USA ","Los Angeles California,USA "]}
van_los=pd.DataFrame.from_dict(van_los).transpose().reset_index()
van_los

,index,0,1
0,start,"Vancouver,CA","Vancouver,CA"
1,stp1,"Seattle Washington,USA","Seattle Washington,USA"
2,stp2,"Portland Oregon,USA","Kennewick Washington,USA"
3,stp3,"Medford Oregon,USA","Twin Falls Idaho,USA"
4,stp4,"Sacramento California,USA","Las Vegas Nevada,USA"
5,Final,"Los Angeles California,USA","Los Angeles California,USA"


In [4]:
van_los['0_lat']=""
van_los['0_lan']=""
van_los['1_lat']=""
van_los['1_lan']=""
van_los=van_los[["index",0,"0_lat","0_lan",1,"1_lat","1_lan"]]
van_los

,index,0,0_lat,0_lan,1,1_lat,1_lan
0,start,"Vancouver,CA",,,"Vancouver,CA",,
1,stp1,"Seattle Washington,USA",,,"Seattle Washington,USA",,
2,stp2,"Portland Oregon,USA",,,"Kennewick Washington,USA",,
3,stp3,"Medford Oregon,USA",,,"Twin Falls Idaho,USA",,
4,stp4,"Sacramento California,USA",,,"Las Vegas Nevada,USA",,
5,Final,"Los Angeles California,USA",,,"Los Angeles California,USA",,


In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [6]:
van_los

,index,0,0_lat,0_lan,1,1_lat,1_lan
0,start,"Vancouver,CA",,,"Vancouver,CA",,
1,stp1,"Seattle Washington,USA",,,"Seattle Washington,USA",,
2,stp2,"Portland Oregon,USA",,,"Kennewick Washington,USA",,
3,stp3,"Medford Oregon,USA",,,"Twin Falls Idaho,USA",,
4,stp4,"Sacramento California,USA",,,"Las Vegas Nevada,USA",,
5,Final,"Los Angeles California,USA",,,"Los Angeles California,USA",,


In [33]:
for i,x,y in zip(van_los.index.values.tolist(),van_los[0],van_los[1]):
    address1 = x
    address2 = y
    geolocator = Nominatim(user_agent="my_application")
    location1 = geolocator.geocode(address1)
    location2 = geolocator.geocode(address2)
    
    van_los["0_lat"][i]=location1.latitude
    van_los["0_lan"][i]=location1.longitude
    
    van_los["1_lat"][i]=location2.latitude
    van_los["1_lan"][i]=location2.longitude

C:\Users\HP\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [34]:
van_los

,index,0,0_lat,0_lan,1,1_lat,1_lan
0,start,"Vancouver,CA",49.2609,-123.114,"Vancouver,CA",49.2609,-123.114
1,stp1,"Seattle Washington,USA",47.6038,-122.33,"Seattle Washington,USA",47.6038,-122.33
2,stp2,"Portland Oregon,USA",45.5202,-122.674,"Kennewick Washington,USA",46.2084,-119.12
3,stp3,"Medford Oregon,USA",42.3264,-122.872,"Twin Falls Idaho,USA",42.5704,-114.46
4,stp4,"Sacramento California,USA",38.5816,-121.494,"Las Vegas Nevada,USA",36.1663,-115.149
5,Final,"Los Angeles California,USA",34.0537,-118.243,"Los Angeles California,USA",34.0537,-118.243


Explore venues at each road

Start the steps of clustering Toronto neighborhood as Newyork example

In [35]:
CLIENT_ID = 'SYNKAEL54LI1HUIAAZJ5VCZT3OISLVZSEAAASQDTUWRCXAVN' # your Foursquare ID
CLIENT_SECRET = 'JSJZWCJ3LNYT0YLLWXZKXXBFZ3JMJGYPX420H2AMWAQJF55K' # your Foursquare Secret
VERSION = '20140715' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SYNKAEL54LI1HUIAAZJ5VCZT3OISLVZSEAAASQDTUWRCXAVN
CLIENT_SECRET:JSJZWCJ3LNYT0YLLWXZKXXBFZ3JMJGYPX420H2AMWAQJF55K


In [251]:
LIMIT=500
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
#         print(venues_list)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
 
    nearby_venues.columns = ['station', 
                  'station Latitude', 
                  'station Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
#     print(nearby_venues.head(2))
    
    return(nearby_venues)

In [252]:
Trip_1_venues = getNearbyVenues(names=van_los[0],
                                 latitudes=van_los['0_lat'],
                                 longitudes=van_los['0_lan'])

In [253]:
Trip_1_venues.shape

(600, 7)

In [256]:
Trip_1_venues.head()

,station,station Latitude,station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Vancouver,CA",49.260872,-123.113953,Whole Foods Market,49.264018,-123.115303,Grocery Store
1,"Vancouver,CA",49.260872,-123.113953,Vij's,49.257525,-123.115083,Indian Restaurant
2,"Vancouver,CA",49.260872,-123.113953,La Taqueria Pinche Taco Shop,49.263534,-123.112795,Taco Place
3,"Vancouver,CA",49.260872,-123.113953,33 Acres Brewing Company,49.263857,-123.105330,Brewery
4,"Vancouver,CA",49.260872,-123.113953,Aperture Coffee Bar,49.263252,-123.110438,Coffee Shop


In [254]:
Trip_2_venues = getNearbyVenues(names=van_los[1],
                                 latitudes=van_los['1_lat'],
                                 longitudes=van_los['1_lan'])

In [255]:
Trip_2_venues.shape

(565, 7)

In [221]:
# one hot encoding
Trip_1_onehot = pd.get_dummies(Trip_1_venues[['Venue Category']], prefix="", prefix_sep="")


# # add neighborhood column back to dataframe
Trip_1_onehot['station'] = Trip_1_venues['station'] 

fixed_columns = [Trip_1_onehot.columns[-1]] + list(Trip_1_onehot.columns[:-1])
Trip_1_onehot = Trip_1_onehot[fixed_columns]

Trip_1_onehot.head()



,station,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Shop,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Marijuana Dispensary,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Motorcycle Shop,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Opera House,Organic Grocery,Other Nightlife,Outdoor Sculpture,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pie Shop,Pizza Place,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Russian Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shoe Store,Soccer Stadium,Social Club,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trade School,Trail,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Yoga Studio
0,"Vancouver,CA",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Vancouver,CA",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Vancouver,CA",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Vancouver,CA",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Vancouver,CA",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [222]:
Trip_2_onehot = pd.get_dummies(Trip_2_venues[['Venue Category']], prefix="", prefix_sep="")


# # add neighborhood column back to dataframe
Trip_2_onehot['station'] = Trip_2_venues['station'] 

fixed_columns = [Trip_2_onehot.columns[-1]] + list(Trip_2_onehot.columns[:-1])
Trip_2_onehot = Trip_2_onehot[fixed_columns]

Trip_2_onehot.head()

,station,ATM,Accessories Store,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Stadium,Beer Bar,Beer Garden,Belgian Restaurant,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Candy Store,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kids Store,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Motorcycle Shop,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Non-Profit,Opera House,Other Nightlife,Outdoor Sculpture,Outlet Mall,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Plaza,Poke Place,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Soccer Stadium,Social Club,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trade School,Trail,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfall,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Vancouver,CA",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Vancouver,CA",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Vancouver,CA",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Vancouver,CA",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Vancouver,CA",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [223]:
Trip_1_grouped = Trip_1_onehot.groupby('station').mean().reset_index()
Trip_1_grouped.head(5)

,station,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Shop,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Marijuana Dispensary,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Motorcycle Shop,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Opera House,Organic Grocery,Other Nightlife,Outdoor Sculpture,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pie Shop,Pizza Place,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Russian Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shoe Store,Soccer Stadium,Social Club,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trade School,Trail,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Yoga Studio
0,"Los Angeles California,USA",0.02,0.01,0.04,0.02,0.00,0.00,0.00,0.01,0.00,0.01,0.04,0.00,0.00,0.00,0.00,0.01,0.00,0.0,0.00,0.03,0.00,0.01,0.03,0.00,0.01,0.00,0.00,0.01,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.05,0.01,0.01,0.00,0.00,0.0,0.01,0.00,0.00,0.00,0.00,0.00,0.0,0.01,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,0.02,0.01,0.00,0.00,0.00,0.01,0.00,0.0,0.00,0.01,0.00,0.00,0.0,0.01,0.05,0.01,0.00,0.00,0.02,0.01,0.02,0.01,0.00,0.00,0.00,0.01,0.01,0.00,0.01,0.00,0.01,0.00,0.01,0.02,0.00,0.00,0.00,0.00,0.01,0.02,0.01,0.02,0.00,0.01,0.00,0.00,0.0,0.01,0.00,0.01,0.00,0.01,0.00,0.03,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.02,0.01,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.03,0.01,0.00,0.0,0.00,0.03,0.00,0.0,0.0,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00
1,"Medford Oregon,USA",0.04,0.00,0.00,0.00,0.05,0.01,0.00,0.01,0.01,0.02,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.0,0.00,0.01,0.01,0.02,0.01,0.00,0.00,0.05,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.01,0.18,0.00,0.00,0.01,0.01,0.0,0.00,0.03,0.00,0.00,0.00,0.00,0.0,0.01,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.01,0.02,0.00,0.01,0.0,0.00,0.00,0.00,0.00,0.0,0.03,0.00,0.00,0.01,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.0,0.03,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.01,0.00,0.00,0.02,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.01,0.00,0.00,0.01,0.00,0.01,0.01,0.00,0.01,0.0,0.03,0.00,0.00,0.0,0.0,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00
2,"Portland Oregon,USA",0.02,0.01

In [187]:
Trip_2_grouped = Trip_2_onehot.groupby('station').mean().reset_index()
Trip_2_grouped.head(5)

In [224]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [225]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['station']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe for trip 1
station_venues_sorted_trip_1 = pd.DataFrame(columns=columns)
station_venues_sorted_trip_1['station'] = Trip_1_grouped['station']

for ind in np.arange(Trip_1_grouped.shape[0]):
    station_venues_sorted_trip_1.iloc[ind, 1:] = return_most_common_venues(Trip_1_grouped.iloc[ind, :],
                                                                           num_top_venues)

# create a new dataframe trip 2
station_venues_sorted_trip_2 = pd.DataFrame(columns=columns)
station_venues_sorted_trip_2['station'] = Trip_2_grouped['station']

for ind in np.arange(Trip_2_grouped.shape[0]):
    station_venues_sorted_trip_2.iloc[ind, 1:] = return_most_common_venues(Trip_2_grouped.iloc[ind, :],
                                                                           num_top_venues)



In [226]:
station_venues_sorted_trip_1.head()

,station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"Los Angeles California,USA",Ice Cream Shop,Coffee Shop,Art Gallery,Bar,Bookstore,Brewery,Plaza,Theater,Sushi Restaurant,Mexican Restaurant,Speakeasy,Japanese Restaurant,Sandwich Place,Italian Restaurant,American Restaurant,Music Venue,New American Restaurant,Gastropub,Art Museum,Museum
1,"Medford Oregon,USA",Coffee Shop,Asian Restaurant,Burger Joint,Sandwich Place,American Restaurant,Mexican Restaurant,Thai Restaurant,Hotel,Department Store,Park,Lingerie Store,Grocery Store,Furniture / Home Store,Fast Food Restaurant,Restaurant,Sporting Goods Shop,Italian Restaurant,Pizza Place,Breakfast Spot,Bakery
2,"Portland Oregon,USA",Coffee Shop,Hotel,Park,Bookstore,Brewery,Food Truck,Sandwich Place,Italian Restaurant,Israeli Restaurant,Grocery Store,French Restaurant,Yoga Studio,Farmers Market,Diner,Concert Hall,Cocktail Bar,Café,Bakery,Dessert Shop,American Restaurant
3,"Sacramento California,USA",American Restaurant,Coffee Shop,Mexican Restaurant,Sandwich Place,Pizza Place,Hotel,Movie Theater,Theater,Chinese Restaurant,Nightclub,Salon / Barbershop,History Museum,Gym,Dessert Shop,Park,Bar,Candy Store,Chocolate Shop,Art Museum,Asian Restaurant
4,"Seattle Washington,USA",Coffee Shop,Cocktail Bar,Vietnamese Restaurant,Hotel,Sandwich Place,Ice Cream Shop,Italian Restaurant,Brewery,Bookstore,French Restaurant,Bakery,Japanese Restaurant,Middle Eastern Restaurant,Café,Seafood Restaurant,Breakfast Spot,Market,Theater,Fish Market,Wine Bar


In [227]:
station_venues_sorted_trip_2.head()

,station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"Kennewick Washington,USA",Convenience Store,American Restaurant,Pizza Place,Fast Food Restaurant,Sandwich Place,Harbor / Marina,Brewery,Mexican Restaurant,Rental Service,Coffee Shop,Pet Store,Park,Outdoor Sculpture,Food & Drink Shop,Food Truck,Lighthouse,Gas Station,Intersection,Grocery Store,Gymnastics Gym
1,"Las Vegas Nevada,USA",Clothing Store,Bar,Coffee Shop,Sporting Goods Shop,Gastropub,Breakfast Spot,Brewery,Women's Store,Steakhouse,Cocktail Bar,Pizza Place,American Restaurant,Boutique,Theater,Mexican Restaurant,Burger Joint,Italian Restaurant,Restaurant,Lounge,Art Gallery
2,"Los Angeles California,USA",Bar,Ice Cream Shop,Coffee Shop,Sushi Restaurant,Bookstore,Brewery,Japanese Restaurant,Mexican Restaurant,Ramen Restaurant,Gastropub,Plaza,French Restaurant,Museum,Breakfast Spot,Sandwich Place,Art Gallery,Art Museum,Athletics & Sports,Seafood Restaurant,Supermarket
3,"Seattle Washington,USA",Coffee Shop,Hotel,Cocktail Bar,Vietnamese Restaurant,Italian Restaurant,Seafood Restaurant,Sushi Restaurant,Bakery,Bookstore,Sandwich Place,Dumpling Restaurant,New American Restaurant,Café,Theater,Art Museum,American Restaurant,Breakfast Spot,Concert Hall,Other Nightlife,Cheese Shop
4,"Twin Falls Idaho,USA",Fast Food Restaurant,Pizza Place,Sandwich Place,Mexican Restaurant,Burger Joint,Coffee Shop,Steakhouse,Bank,Sports Bar,Pharmacy,Sushi Restaurant,Grocery Store,Burrito Place,Discount Store,Hotel,Convenience Store,Department Store,Park,Italian Restaurant,Clothing Store


In [228]:
x=station_venues_sorted_trip_1
y=station_venues_sorted_trip_2
station_venues_sorted_total=pd.concat([x, y], ignore_index=True)
station_venues_sorted_total.drop_duplicates(subset="station", keep='first', inplace=True)

In [229]:
station_venues_sorted_total

,station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"Los Angeles California,USA",Ice Cream Shop,Coffee Shop,Art Gallery,Bar,Bookstore,Brewery,Plaza,Theater,Sushi Restaurant,Mexican Restaurant,Speakeasy,Japanese Restaurant,Sandwich Place,Italian Restaurant,American Restaurant,Music Venue,New American Restaurant,Gastropub,Art Museum,Museum
1,"Medford Oregon,USA",Coffee Shop,Asian Restaurant,Burger Joint,Sandwich Place,American Restaurant,Mexican Restaurant,Thai Restaurant,Hotel,Department Store,Park,Lingerie Store,Grocery Store,Furniture / Home Store,Fast Food Restaurant,Restaurant,Sporting Goods Shop,Italian Restaurant,Pizza Place,Breakfast Spot,Bakery
2,"Portland Oregon,USA",Coffee Shop,Hotel,Park,Bookstore,Brewery,Food Truck,Sandwich Place,Italian Restaurant,Israeli Restaurant,Grocery Store,French Restaurant,Yoga Studio,Farmers Market,Diner,Concert Hall,Cocktail Bar,Café,Bakery,Dessert Shop,American Restaurant
3,"Sacramento California,USA",American Restaurant,Coffee Shop,Mexican Restaurant,Sandwich Place,Pizza Place,Hotel,Movie Theater,Theater,Chinese Restaurant,Nightclub,Salon / Barbershop,History Museum,Gym,Dessert Shop,Park,Bar,Candy Store,Chocolate Shop,Art Museum,Asian Restaurant
4,"Seattle Washington,USA",Coffee Shop,Cocktail Bar,Vietnamese Restaurant,Hotel,Sandwich Place,Ice Cream Shop,Italian Restaurant,Brewery,Bookstore,French Restaurant,Bakery,Japanese Restaurant,Middle Eastern Restaurant,Café,Seafood Restaurant,Breakfast Spot,Market,Theater,Fish Market,Wine Bar
5,"Vancouver,CA",Coffee Shop,Bakery,Park,Seafood Restaurant,Hotel,Café,BBQ Joint,Italian Restaurant,Ice Cream Shop,Brewery,Trail,Vegetarian / Vegan Restaurant,Pizza Place,French Restaurant,Concert Hall,Middle Eastern Restaurant,Mexican Restaurant,Dessert Shop,Taco Place,Japanese Restaurant
6,"Kennewick Washington,USA",Convenience Store,American Restaurant,Pizza Place,Fast Food Restaurant,Sandwich Place,Harbor / Marina,Brewery,Mexican Restaurant,Rental Service,Coffee Shop,Pet Store,Park,Outdoor Sculpture,Food & Drink Shop,Food Truck,Lighthouse,Gas Station,Intersection,Grocery Store,Gymnastics Gym
7,"Las Vegas Nevada,USA",Clothing Store,Bar,Coffee Shop,Sporting Goods Shop,Gastropub,Breakfast Spot,Brewery,Women's Store,Steakhouse,Cocktail Bar,Pizza Place,American Restaurant,Boutique,Theater,Mexican Restaurant,Burger Joint,Italian Restaurant,Restaurant,Lounge,Art Gallery
10,"Twin Falls Idaho,USA",Fast Food Restaurant,Pizza Place,Sandwich Place,Mexican Restaurant,Burger Joint,Coffee Shop,Steakhouse,Bank,Sports Bar,Pharmacy,Sushi Restaurant,Grocery Store,Burrito Place,Discount Store,Hotel,Convenience Store,Department Store,Park,Italian Restaurant,Clothing Store


<h4> Clustring </h4>

In [230]:
total_Trip_grouped=pd.concat([Trip_1_grouped, Trip_2_grouped], ignore_index=True)
total_Trip_grouped=total_Trip_grouped.fillna(0)
fixed_columns = [total_Trip_grouped.columns[-1]] + list(total_Trip_grouped.columns[:-1])
total_Trip_grouped = total_Trip_grouped[fixed_columns]


C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [231]:
total_Trip_grouped.drop_duplicates(subset="station", keep='first', inplace=True)

In [232]:
total_Trip_grouped

,station,ATM,Accessories Store,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Motorcycle Shop,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Opera House,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Plaza,Poke Place,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Soccer Stadium,Social Club,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trade School,Trail,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Los Angeles California,USA",0.000,0.00,0.020000,0.01,0.04,0.02,0.0,0.00,0.00,0.00,0.01,0.00,0.010,0.000000,0.040,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.0,0.03,0.00,0.000,0.01,0.03,0.00,0.01,0.000000,0.000000,0.000000,0.010000,0.01,0.01,0.0,0.00,0.01,0.000000,0.00,0.00,0.000000,0.00,0.050000,0.0,0.01,0.01,0.000000,0.00,0.00,0.01,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.000000,0.01,0.00,0.00,0.0,0.000,0.00,0.010,0.01,0.01,0.000000,0.00,0.00,0.000000,0.02,0.0,0.01,0.00,0.00,0.000000,0.01,0.000000,0.00,0.000,0.00,0.000000,0.00,0.01,0.00,0.00,0.00,0.00,0.010000,0.00,0.050000,0.01,0.0,0.0,0.000,0.00,0.00,0.020000,0.01,0.02,0.01,0.00,0.00,0.00,0.00,0.01,0.01,0.000,0.00,0.01,0.00,0.0,0.00,0.01,0.00,0.01,0.020000,0.00,0.00,0.00,0.00,0.01,0.02,0.01,0.02,0.00,0.01,0.00,0.00,0.000,0.0,0.00,0.010000,0.00,0.01,0.00,0.000,0.000000,0.00,0.01,0.000000,0.03,0.00,0.000000,0.00,0.01,0.00,0.000000,0.000,0.010000,0.00,0.00,0.0,0.00,0.020000,0.01,0.01,0.00,0.01,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.000000,0.000000,0.00,0.01,0.000000,0.00,0.000000,0.0

In [233]:
kclusters = 4

total_Trip_grouped_clustering = total_Trip_grouped.drop('station', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(total_Trip_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 0, 3, 0, 0, 1, 3, 2])

In [234]:
van_los

,index,0,0_lat,0_lan,1,1_lat,1_lan
0,start,"Vancouver,CA",49.2609,-123.114,"Vancouver,CA",49.2609,-123.114
1,stp1,"Seattle Washington,USA",47.6038,-122.33,"Seattle Washington,USA",47.6038,-122.33
2,stp2,"Portland Oregon,USA",45.5202,-122.674,"Kennewick Washington,USA",46.2084,-119.12
3,stp3,"Medford Oregon,USA",42.3264,-122.872,"Twin Falls Idaho,USA",42.5704,-114.46
4,stp4,"Sacramento California,USA",38.5816,-121.494,"Las Vegas Nevada,USA",36.1663,-115.149
5,Final,"Los Angeles California,USA",34.0537,-118.243,"Los Angeles California,USA",34.0537,-118.243


In [235]:
df1=van_los.drop([1, '1_lat','1_lan'], axis=1)
df1.rename(columns={0:"station","0_lat":"latitude","0_lan":"longitude"},inplace=True)

In [236]:
df2=van_los.drop([0, '0_lat','0_lan'], axis=1)
df2.rename(columns={1:"station","1_lat":"latitude","1_lan":"longitude"},inplace=True)

In [237]:
df_total=pd.concat([df1, df2],keys=['1st_road', '2nd_road'])

In [238]:
df_total

index                      station latitude longitude
1st_road 0  start                 Vancouver,CA  49.2609  -123.114
         1   stp1      Seattle Washington,USA   47.6038   -122.33
         2   stp2         Portland Oregon,USA   45.5202  -122.674
         3   stp3          Medford Oregon,USA   42.3264  -122.872
         4   stp4   Sacramento California,USA   38.5816  -121.494
         5  Final  Los Angeles California,USA   34.0537  -118.243
2nd_road 0  start                 Vancouver,CA  49.2609  -123.114
         1   stp1      Seattle Washington,USA   47.6038   -122.33
         2   stp2     Kennewick Washington,USA  46.2084   -119.12
         3   stp3        Twin Falls Idaho,USA   42.5704   -114.46
         4   stp4        Las Vegas Nevada,USA   36.1663  -115.149
         5  Final  Los Angeles California,USA   34.0537  -118.243

In [239]:
# add clustering labels
# total_Trip_grouped.drop('Cluster Labels',axis=1,inplace= True)
total_Trip_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
total_trip_merged = df_total[["station","latitude","longitude"]]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
total_trip_merged = total_trip_merged.join(total_Trip_grouped.set_index('station'), on='station')



In [240]:
total_trip_merged.drop_duplicates(subset="station", keep='first', inplace=True)

In [241]:
total_trip_merged_cluster=total_trip_merged.groupby('Cluster Labels')

In [242]:
# create map
latitude=42.234615 # mean latitude between the start and end of trip
longitude=-123.984943 # mean the longitude between the start and end of the trip

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(total_trip_merged['latitude'], total_trip_merged['longitude'], 
                                  total_trip_merged['station'], total_trip_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=35,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

Examin Clusters

In [243]:
total_Trip_grouped
cluster=total_Trip_grouped[['Cluster Labels','station']]
cluster

,Cluster Labels,station
0,0,"Los Angeles California,USA"
1,3,"Medford Oregon,USA"
2,0,"Portland Oregon,USA"
3,3,"Sacramento California,USA"
4,0,"Seattle Washington,USA"
5,0,"Vancouver,CA"
6,1,"Kennewick Washington,USA"
7,3,"Las Vegas Nevada,USA"
10,2,"Twin Falls Idaho,USA"


In [244]:
station_venues_sorted_total.head()

,station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"Los Angeles California,USA",Ice Cream Shop,Coffee Shop,Art Gallery,Bar,Bookstore,Brewery,Plaza,Theater,Sushi Restaurant,Mexican Restaurant,Speakeasy,Japanese Restaurant,Sandwich Place,Italian Restaurant,American Restaurant,Music Venue,New American Restaurant,Gastropub,Art Museum,Museum
1,"Medford Oregon,USA",Coffee Shop,Asian Restaurant,Burger Joint,Sandwich Place,American Restaurant,Mexican Restaurant,Thai Restaurant,Hotel,Department Store,Park,Lingerie Store,Grocery Store,Furniture / Home Store,Fast Food Restaurant,Restaurant,Sporting Goods Shop,Italian Restaurant,Pizza Place,Breakfast Spot,Bakery
2,"Portland Oregon,USA",Coffee Shop,Hotel,Park,Bookstore,Brewery,Food Truck,Sandwich Place,Italian Restaurant,Israeli Restaurant,Grocery Store,French Restaurant,Yoga Studio,Farmers Market,Diner,Concert Hall,Cocktail Bar,Café,Bakery,Dessert Shop,American Restaurant
3,"Sacramento California,USA",American Restaurant,Coffee Shop,Mexican Restaurant,Sandwich Place,Pizza Place,Hotel,Movie Theater,Theater,Chinese Restaurant,Nightclub,Salon / Barbershop,History Museum,Gym,Dessert Shop,Park,Bar,Candy Store,Chocolate Shop,Art Museum,Asian Restaurant
4,"Seattle Washington,USA",Coffee Shop,Cocktail Bar,Vietnamese Restaurant,Hotel,Sandwich Place,Ice Cream Shop,Italian Restaurant,Brewery,Bookstore,French Restaurant,Bakery,Japanese Restaurant,Middle Eastern Restaurant,Café,Seafood Restaurant,Breakfast Spot,Market,Theater,Fish Market,Wine Bar


In [245]:
cluster_venues_sorted_total=pd.merge(station_venues_sorted_total,cluster, on='station')
l=cluster_venues_sorted_total

In [246]:

l.loc[l['Cluster Labels'] == 0,l.columns[[21]+ list(range(0, station_venues_sorted_total.shape[1]-1))]]

,Cluster Labels,station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue
0,0,"Los Angeles California,USA",Ice Cream Shop,Coffee Shop,Art Gallery,Bar,Bookstore,Brewery,Plaza,Theater,Sushi Restaurant,Mexican Restaurant,Speakeasy,Japanese Restaurant,Sandwich Place,Italian Restaurant,American Restaurant,Music Venue,New American Restaurant,Gastropub,Art Museum
2,0,"Portland Oregon,USA",Coffee Shop,Hotel,Park,Bookstore,Brewery,Food Truck,Sandwich Place,Italian Restaurant,Israeli Restaurant,Grocery Store,French Restaurant,Yoga Studio,Farmers Market,Diner,Concert Hall,Cocktail Bar,Café,Bakery,Dessert Shop
4,0,"Seattle Washington,USA",Coffee Shop,Cocktail Bar,Vietnamese Restaurant,Hotel,Sandwich Place,Ice Cream Shop,Italian Restaurant,Brewery,Bookstore,French Restaurant,Bakery,Japanese Restaurant,Middle Eastern Restaurant,Café,Seafood Restaurant,Breakfast Spot,Market,Theater,Fish Market
5,0,"Vancouver,CA",Coffee Shop,Bakery,Park,Seafood Restaurant,Hotel,Café,BBQ Joint,Italian Restaurant,Ice Cream Shop,Brewery,Trail,Vegetarian / Vegan Restaurant,Pizza Place,French Restaurant,Concert Hall,Middle Eastern Restaurant,Mexican Restaurant,Dessert Shop,Taco Place


In [247]:
l.loc[l['Cluster Labels'] == 1,l.columns[[21]+ list(range(0, station_venues_sorted_total.shape[1]-1))]]

,Cluster Labels,station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue
6,1,"Kennewick Washington,USA",Convenience Store,American Restaurant,Pizza Place,Fast Food Restaurant,Sandwich Place,Harbor / Marina,Brewery,Mexican Restaurant,Rental Service,Coffee Shop,Pet Store,Park,Outdoor Sculpture,Food & Drink Shop,Food Truck,Lighthouse,Gas Station,Intersection,Grocery Store


In [248]:
l.loc[l['Cluster Labels'] == 2,l.columns[[21]+ list(range(0, station_venues_sorted_total.shape[1]-1))]]

,Cluster Labels,station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue
8,2,"Twin Falls Idaho,USA",Fast Food Restaurant,Pizza Place,Sandwich Place,Mexican Restaurant,Burger Joint,Coffee Shop,Steakhouse,Bank,Sports Bar,Pharmacy,Sushi Restaurant,Grocery Store,Burrito Place,Discount Store,Hotel,Convenience Store,Department Store,Park,Italian Restaurant


In [249]:
l.loc[l['Cluster Labels'] == 3,l.columns[[21]+ list(range(0, station_venues_sorted_total.shape[1]-1))]]

,Cluster Labels,station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue
1,3,"Medford Oregon,USA",Coffee Shop,Asian Restaurant,Burger Joint,Sandwich Place,American Restaurant,Mexican Restaurant,Thai Restaurant,Hotel,Department Store,Park,Lingerie Store,Grocery Store,Furniture / Home Store,Fast Food Restaurant,Restaurant,Sporting Goods Shop,Italian Restaurant,Pizza Place,Breakfast Spot
3,3,"Sacramento California,USA",American Restaurant,Coffee Shop,Mexican Restaurant,Sandwich Place,Pizza Place,Hotel,Movie Theater,Theater,Chinese Restaurant,Nightclub,Salon / Barbershop,History Museum,Gym,Dessert Shop,Park,Bar,Candy Store,Chocolate Shop,Art Museum
7,3,"Las Vegas Nevada,USA",Clothing Store,Bar,Coffee Shop,Sporting Goods Shop,Gastropub,Breakfast Spot,Brewery,Women's Store,Steakhouse,Cocktail Bar,Pizza Place,American Restaurant,Boutique,Theater,Mexican Restaurant,Burger Joint,Italian Restaurant,Restaurant,Lounge


In [250]:
l.loc[l['Cluster Labels'] == 4,l.columns[[21]+ list(range(0, station_venues_sorted_total.shape[1]-1))]]

,Cluster Labels,station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue
